In [82]:
import pandas as pd
import numpy as np
import tempfile
import os
import sqlite3

nazwy musza miec liczbe pojedyncza

In [83]:
Badge = pd.read_csv("Badges.csv", error_bad_lines=False, comment="#")
Comment = pd.read_csv("Comments.csv", error_bad_lines=False, comment="#")
PostLink = pd.read_csv("PostLinks.csv", error_bad_lines=False, comment="#")
Post = pd.read_csv("Posts.csv", error_bad_lines=False, comment="#")
Tag = pd.read_csv("Tags.csv", error_bad_lines=False, comment="#")
User = pd.read_csv("Users.csv", error_bad_lines=False, comment="#")
Vote = pd.read_csv("Votes.csv", error_bad_lines=False, comment="#")

In [84]:
base = os.path.join(tempfile.mkdtemp(), "base.db")
if os.path.isfile(base):
    os.remove(base)
conn = sqlite3.connect(base)

In [85]:
Badge.to_sql("Badge", conn, index = False)
Comment.to_sql("Comment", conn, index = False)
PostLink.to_sql("PostLink", conn, index = False)
Post.to_sql("Post", conn, index = False)
Tag.to_sql("Tag", conn, index = False)
User.to_sql("User", conn, index = False)
Vote.to_sql("Vote", conn, index = False)

In [86]:
df_sql_1 = pd.read_sql_query("""
SELECT PostId, COUNT(*) AS UpVotes
FROM Vote
WHERE VoteTypeId=2
GROUP BY PostId
""", conn)

In [87]:
df_pd_1 = Vote.loc[(Vote.VoteTypeId == 2)].groupby("PostId").size().reset_index(name = "UpVotes")

In [88]:
type(df_sql_1)
type(df_pd_1)

pandas.core.frame.DataFrame

In [89]:
df_sql_1.equals(df_pd_1)

True

In [90]:
df_sql_2 = pd.read_sql_query("""
SELECT Title, Score, ViewCount, FavoriteCount
FROM Post
WHERE PostTypeId=1 AND FavoriteCount >= 25 AND ViewCount>=10000
""", conn)

In [100]:
df_pd_2 = Post.loc[(Post.PostTypeId == 1) & (Post.FavoriteCount >= 25) & (Post.ViewCount >= 10000), ["Title", "Score", "ViewCount", "FavoriteCount"]].reset_index(drop = True)

In [96]:
df_sql_2.equals(df_pd_2)

True

In [122]:
df_sql_3 = pd.read_sql_query("""
SELECT Tag.TagName, Tag.Count, Post.OwnerUserId,
User.Location, User.DisplayName
FROM Tag
JOIN Post ON Post.Id=Tag.WikiPostId
JOIN User ON User.AccountId=Post.OwnerUserId
WHERE OwnerUserId != -1
ORDER BY Count DESC
""", conn)

In [155]:
t = pd.merge(pd.merge(Tag, Post, left_on = "WikiPostId", right_on = "Id"), User, left_on = "OwnerUserId", right_on = "AccountId")
df_pd_3 = t.loc[(t.OwnerUserId != -1), ["TagName", "Count", "OwnerUserId", "Location", "DisplayName"]].sort_values("Count", ascending = False).reset_index(drop = True)

In [156]:
df_sql_3.equals(df_pd_3)

True

In [144]:
df_sql_4 = pd.read_sql_query("""
SELECT Post.Title, RelatedTab.NumLinks FROM
    (SELECT RelatedPostId AS PostId, COUNT(*) AS NumLinks
    FROM PostLink GROUP BY RelatedPostId) AS RelatedTab
JOIN Post ON RelatedTab.PostId=Post.Id
WHERE Post.PostTypeId=1
ORDER BY NumLinks DESC
""", conn)

In [159]:
RelatedTab = PostLink.groupby("RelatedPostId").size().reset_index(name = "NumLinks").rename(columns = {"RelatedPostId": "PostId"})
t = pd.merge(RelatedTab, Post, left_on = "PostId", right_on = "Id")
df_pd_4 = t.loc[(t.PostTypeId == 1), ["Title", "NumLinks"]].sort_values("NumLinks", ascending = False).reset_index(drop = True)

In [177]:
df_sql_4.sort_values("Title").reset_index(drop = True).equals(df_pd_4.sort_values("Title").reset_index(drop = True))

True

In [189]:
df_sql_5 = pd.read_sql_query("""
SELECT UpVotesTab.*, Post.Title FROM
    (SELECT PostId, COUNT(*) AS UpVotes FROM Vote WHERE VoteTypeId=2 GROUP BY PostId) AS UpVotesTab
JOIN Post ON UpVotesTab.PostId=Post.Id
WHERE Post.PostTypeId=1
ORDER BY UpVotesTab.UpVotes DESC
""", conn)

In [187]:
UpVotesTab = Vote.loc[(Vote.VoteTypeId == 2)].groupby("PostId").size().reset_index(name = "UpVotes")
t = pd.merge(UpVotesTab, Post, left_on = "PostId", right_on = "Id")
df_pd_5 = t.loc[(t.PostTypeId == 1), ["PostId", "UpVotes", "Title"]].sort_values("UpVotes", ascending = False).reset_index(drop = True)

In [188]:
df_sql_5.sort_values("Title").reset_index(drop = True).equals(df_pd_5.sort_values("Title").reset_index(drop = True))

True

In [216]:
df_sql_6 = pd.read_sql_query("""
SELECT UpVotesTab.PostId, UpVotesTab.UpVotes, IFNULL(DownVotesTab.DownVotes, 0) AS DownVotes FROM
    (SELECT PostId, COUNT(*) AS UpVotes FROM Vote 
    WHERE VoteTypeId=2 GROUP BY PostId) AS UpVotesTab
LEFT JOIN
    (SELECT PostId, COUNT(*) AS DownVotes FROM Vote
    WHERE VoteTypeId=3 GROUP BY PostId) AS DownVotesTab
ON UpVotesTab.PostId=DownVotesTab.PostId
""", conn)

In [214]:
UpVotesTab = Vote.loc[(Vote.VoteTypeId == 2)].groupby("PostId").size().reset_index(name = "UpVotes")
DownVotesTab = Vote.loc[(Vote.VoteTypeId == 3)].groupby("PostId").size().reset_index(name = "UpVotes")
df_pd_6 = pd.merge(UpVotesTab, DownVotesTab, on = "PostId", how = "left").fillna(0).astype(int).rename(columns = {"UpVotes_x": "UpVotes", "UpVotes_y": "DownVotes"})

In [217]:
df_sql_6.equals(df_pd_6)

True

In [72]:
df_sql_7 = pd.read_sql_query("""
SELECT PostId, UpVotes-DownVotes AS Vote FROM 
    (SELECT UpVotesTab.PostId, UpVotesTab.UpVotes, IFNULL(DownVotesTab.DownVotes, 0) AS DownVotes FROM
        (SELECT PostId, COUNT(*) AS UpVotes FROM Vote
        WHERE VoteTypeId=2 GROUP BY PostId) AS UpVotesTab
    LEFT JOIN
        (SELECT PostId, COUNT(*) AS DownVotes
        FROM Vote WHERE VoteTypeId=3 GROUP BY PostId) AS DownVotesTab
    ON UpVotesTab.PostId=DownVotesTab.PostId
    UNION
    SELECT DownVotesTab.PostId, IFNULL(UpVotesTab.UpVotes, 0) AS UpVotes, DownVotesTab.DownVotes FROM
        (SELECT PostId, COUNT(*) AS DownVotes FROM Vote
        WHERE VoteTypeId=3 GROUP BY PostId) AS DownVotesTab
    LEFT JOIN
        (SELECT PostId, COUNT(*) AS UpVotes FROM Vote
        WHERE VoteTypeId=2 GROUP BY PostId) AS UpVotesTab
    ON DownVotesTab.PostId=UpVotesTab.PostId)
""", conn)
df_sql_7

,PostId,Vote
0,1,42
1,2,13
2,3,10
3,4,16
4,5,31
...,...,...
45704,63933,2
45705,63934,1
45706,63939,1
45707,63940,1
